In [1]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator, root_validator
from typing import List

import openai
import os
openai.organization = os.getenv('OPENAI_ORG')
openai.api_key = os.getenv('OPENAI_API_KEY')

from rich import print
import ast

In [2]:
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [3]:
class CustomObj(BaseModel):
    
    python_code: str = Field(description="python code")

    @root_validator()
    def verify_python_code(cls, values):
        python_code = values.get("python_code")
        try:
            ast.parse(python_code)
        except:
            raise ValueError("The python code is invalid.")
        return values

In [4]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=CustomObj)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"python_code": {"title": "Python Code", "description": "python code", "type": "string"}}, "required": ["python_code"]}\n```'

In [5]:
query_prefix = '''
Given the following high level leetcode problem description, write a short Python code snippet that solves the 
problem. Please provide the function name if the snippet is a function.

Problem Description:
{description}
'''

query = query_prefix.format(description='Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.')
query

'\nGiven the following high level leetcode problem description, write a short Python code snippet that solves the \nproblem. Please provide the function name if the snippet is a function.\n\nProblem Description:\nGiven a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.\n'

In [6]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)
prompt

PromptTemplate(input_variables=['query'], output_parser=None, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"python_code": {"title": "Python Code", "description": "python code", "type": "string"}}, "required": ["python_code"]}\n```'}, template='Answer the user query.\n{format_instructions}\n{query}\n', template_format='f-string', validate_template=True)

In [7]:
input_prompt = prompt.format_prompt(query=query).to_string()
input_prompt

'Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"python_code": {"title": "Python Code", "description": "python code", "type": "string"}}, "required": ["python_code"]}\n```\n\nGiven the following high level leetcode problem description, write a short Python code snippet that solves the \nproblem. Please provide the function name if the snippet is a function.\n\nProblem Description:\nGiven a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.\n\n'

In [8]:
output = model(input_prompt)
output

'\nOutput:\n{"python_code": "def longestPalindrome(s):\\n    n = len(s)\\n    dp = [[False for _ in range(n)] for _ in range(n)]\\n    ans = \'\'\\n    for i in range(n-1, -1, -1):\\n        for j in range(i, n):\\n            dp[i][j] = s[i] == s[j] and (j - i < 3 or dp[i+1][j-1])\\n            if dp[i][j] and j - i + 1 > len(ans):\\n                ans = s[i:j+1]\\n    return ans"}'

In [9]:
custom_obj = parser.parse(output)
print(custom_obj.python_code)

def longestPalindrome(s):
    n = len(s)
    dp = [[False for _ in range(n)] for _ in range(n)]
    ans = ''
    for i in range(n-1, -1, -1):
        for j in range(i, n):
            dp = s == s and (j - i < 3 or dp)
            if dp and j - i + 1 > len(ans):
                ans = s
    return ans

In [10]:
try:
    exec(custom_obj.python_code)
    print("Success!")
except Exception as e:
    print("Failed!")

Success!